In [1]:
from PIL import Image
import requests
from transformers import AutoProcessor, Kosmos2ForConditionalGeneration

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Kosmos2ForConditionalGeneration.from_pretrained("microsoft/kosmos-2-patch14-224").to("cuda")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
image = Image.open("../media/input_images/103 53.png")
prompt = "<grounding>Describe this pattern in background features:"

inputs = processor(text=prompt, images=image, return_tensors="pt")
inputs.to("cuda")
print(inputs.keys())
print("Image embeds:", inputs["image_embeds_position_mask"])

generated_ids = model.generate(
    pixel_values=inputs["pixel_values"],
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    image_embeds=None,
    image_embeds_position_mask=inputs["image_embeds_position_mask"],
    use_cache=True,
    max_new_tokens=64,
)
print(generated_ids.shape)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
processed_text = processor.post_process_generation(generated_text, cleanup_and_extract=False)
print(processed_text)

caption, entities = processor.post_process_generation(generated_text)
print(caption)

dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'image_embeds_position_mask'])
Image embeds: tensor([[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]], device='cuda:0')
torch.Size([1, 104])
<grounding> Describe this pattern in background features:<phrase> A large floral bouquet</phrase><object><patch_index_0529><patch_index_0793></object> of red roses and<phrase> green leaves</phrase><object><patch_index_0000><patch_index_0333></object> on a white background.
Describe this pattern in background features: A large floral bouquet of red roses and green leaves on a white background.


In [ ]:
entities